In [1]:
import json
import sys
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime
import pytz
import time
from joblib import load
import logging
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import roll_time_series, impute
import talib
import os
import threading

logging.basicConfig(filename='trading_USDCAD_Buy_Sell_D1.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def init_mt5_connection(login, password, server):
    if not mt5.initialize(login=login, password=password, server=server):
        logging.error(f"initialize() failed, error code = {mt5.last_error()}")
        sys.exit()
    logging.info("Connected to MetaTrader 5")
    print("Connected to MetaTrader 5")

def fetch_latest_candle(symbol, timeframe):
    latest_candles = mt5.copy_rates_from_pos(symbol, timeframe, 0, 1)
    return latest_candles[0] if len(latest_candles) > 0 else None

def fetch_historical_data(symbol, timeframe, start_date, end_date):
    logging.info(f"Fetching historical data for {symbol}")
    data = mt5.copy_rates_range(symbol, timeframe, start_date, end_date)
    ohlc_data = pd.DataFrame(data)
    ohlc_data['time'] = pd.to_datetime(ohlc_data['time'], unit='s')
    return ohlc_data[['time', 'open', 'high', 'low', 'close']]

def add_rolling_features(df, window):
    logging.info("Adding rolling features")
    df['rolling_mean_open'] = df['open'].rolling(window=window).mean()
    df['rolling_std_open'] = df['open'].rolling(window=window).std()
    df['rolling_mean_close'] = df['close'].rolling(window=window).mean()
    df['rolling_std_close'] = df['close'].rolling(window=window).std()
    df['rolling_mean_high'] = df['high'].rolling(window=window).mean()
    df['rolling_std_high'] = df['high'].rolling(window=window).std()
    df['rolling_mean_low'] = df['low'].rolling(window=window).mean()
    df['rolling_std_low'] = df['low'].rolling(window=window).std()
    return df

def add_lag_features(df, lags):
    logging.info("Adding lag features")
    for lag in lags:
        df[f'open_lag_{lag}'] = df['open'].shift(lag)
        df[f'close_lag_{lag}'] = df['close'].shift(lag)
        df[f'high_lag_{lag}'] = df['high'].shift(lag)
        df[f'low_lag_{lag}'] = df['low'].shift(lag)
    return df

def calculate_indicators(df):
    logging.info("Calculating indicators")
    for period in [15, 23, 42, 145]:
        df[f'WILLR_{period}'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=period)
    return df

def align_features_with_json(df, features_json):
    expected_features = features_json
    missing_features = [feat for feat in expected_features if feat not in df.columns]
    extra_features = [feat for feat in df.columns if feat not in expected_features]
    
    logging.info(f"Missing features: {missing_features}")
    logging.info(f"Extra features: {extra_features}")
    
    for feat in missing_features:
        df[feat] = 0

    df = df[expected_features]
    
    return df

def process_data_for_features(df, symbol, signals, selected_features):
    logging.info(f"Processing data for feature extraction for {symbol}")
    X_combined = pd.DataFrame(index=df['time'])
    
    for signal in signals:
        df_melted = df[['time', signal]].copy()
        df_melted["Symbols"] = symbol
        df_rolled = roll_time_series(df_melted, column_id="Symbols", column_sort="time", max_timeshift=20, min_timeshift=5)
        X = extract_features(df_rolled.drop("Symbols", axis=1), column_id="id", column_sort="time", column_value=signal, impute_function=impute, show_warnings=False)
        X = X.set_index(X.index.map(lambda x: x[1]), drop=True)
        selected_features_X = [feature for feature in selected_features if feature in X.columns]
        X_filtered = X[selected_features_X]
        X_combined = X_combined.merge(X_filtered, left_index=True, right_index=True, how='left')
        X_combined = X_combined.dropna()
    return X_combined

def fetch_and_process_data(symbol, timeframe, signals, selected_features, start_date, end_date):
    logging.info(f"Fetching and processing data for {symbol}")
    df = fetch_historical_data(symbol, timeframe, start_date, end_date)
    df = calculate_indicators(df)
    df = add_rolling_features(df, window=5)
    df = add_lag_features(df, lags=[1, 2, 3, 4, 5])
    
    logging.info(f"NaNs before drop for {symbol}: {df.isnull().sum().sum()}")
    df = df.dropna().reset_index(drop=True)
    logging.info(f"NaNs after drop for {symbol}: {df.isnull().sum().sum()}")

    features = process_data_for_features(df, symbol, signals, selected_features)
    df = df.set_index('time')
    combined_df = df.merge(features, left_index=True, right_index=True, how='left')
    combined_df = combined_df[selected_features]
    logging.info(f"NaNs after merging for {symbol}: {combined_df.isnull().sum().sum()}")
    combined_df = combined_df.dropna()
    logging.info(f"Final NaNs for {symbol}: {combined_df.isnull().sum().sum()}")
    
    return combined_df

def calculate_prices(entry_price, risk_reward_ratio, mean_candle_size):
    logging.info("Calculating SL and TP prices")
    risk_part, reward_part = map(int, risk_reward_ratio.split(':'))
    risk_amount = mean_candle_size * risk_part
    reward_amount = mean_candle_size * reward_part
    sl_price = entry_price - risk_amount
    tp_price = entry_price + reward_amount
    return sl_price, tp_price

def place_order(symbol, volume, sl_price, tp_price, trade_type):
    logging.info(f"Placing {trade_type} order for {symbol}")
    
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None or not symbol_info.visible or symbol_info.trade_mode == mt5.SYMBOL_TRADE_MODE_DISABLED:
        logging.error(f"Market for {symbol} is closed or unavailable.")
        print(f"Market for {symbol} is closed. Cannot place order.")
        return

    price = mt5.symbol_info_tick(symbol).ask if trade_type == "Buy" else mt5.symbol_info_tick(symbol).bid
    order_type = mt5.ORDER_TYPE_BUY if trade_type == "Buy" else mt5.ORDER_TYPE_SELL
    
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": volume,
        "type": order_type,
        "price": price,
        "sl": sl_price,
        "tp": tp_price,
        "deviation": 10,
        "magic": 0,
        "comment": f"Python script {trade_type} order",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    
    result = mt5.order_send(request)
    
    if result and result.retcode == mt5.TRADE_RETCODE_DONE:
        logging.info(f"{trade_type} order placed successfully for {symbol}.")
        print(f"{trade_type} order placed successfully for {symbol}.")
    else:
        logging.error(f"Order placement failed for {symbol}: {result.retcode} - {mt5.last_error()}")
        print(f"Order placement failed for {symbol}: {result.retcode} - {mt5.last_error()}")

def apply_custom_threshold(probas, custom_threshold):
    return (probas >= custom_threshold).astype(int)

def predict_and_trade(scaler, model, symbol, volume, timeframe, risk_reward_ratio, mean_candle_size, start_date, end_date, features_json, trade_type, threshold=None):
    logging.info(f"Starting prediction and trading process for {symbol} - {trade_type}")
    signals = ['WILLR_15', 'WILLR_42']
    df_processed = fetch_and_process_data(symbol, timeframe, signals, features_json, start_date, end_date)
    original_index = df_processed.index
    df_processed = df_processed.shift(periods=1, axis=0).dropna()
    df_processed.index = original_index
    df_processed = align_features_with_json(df_processed, features_json)
    logging.info(f"Number of features for {symbol} - {trade_type}: {df_processed.shape[1]}")
    scaled_data = scaler.transform(df_processed)
    probas = model.predict_proba(scaled_data)[:, 1]
    custom_threshold = threshold if threshold is not None else 0.5
    y_pred = apply_custom_threshold(probas, custom_threshold)
    
    df_pred = pd.DataFrame(index=df_processed.index)
    df_pred['prediction'] = y_pred
    save_predictions_to_csv(df_pred, symbol, trade_type)
    
    if len(y_pred) > 1 and y_pred[-1] == 1:
        entry_price = mt5.symbol_info_tick(symbol).ask if trade_type == "Buy" else mt5.symbol_info_tick(symbol).bid
        sl_price, tp_price = calculate_prices(entry_price, risk_reward_ratio, mean_candle_size)
        place_order(symbol, volume, sl_price, tp_price, trade_type)
    else:
        logging.info(f"No {trade_type} signal generated for {symbol}. Doing nothing.")
        print(f"No {trade_type} signal generated for {symbol}. Doing nothing.")

def save_predictions_to_csv(df_pred, symbol, trade_type):
    logging.info(f"Saving {trade_type} predictions to CSV for {symbol}")
    file_path = f'predict_{symbol}_D1_3112_{trade_type}.csv'
    if not os.path.isfile(file_path):
        df_pred.to_csv(file_path, index=True)
    else:
        df_pred.to_csv(file_path, mode='a', header=False, index=True)

def process_pair(config, utc_from, utc_to):
    logging.info(f"Processing pair: {config['symbol']} for Buy and Sell")
    buyscaler, buymodel, sellscaler, sellmodel = None, None, None, None
    try:
        buyscaler = load(config["buy_scaler_path"])
        buymodel = load(config["buy_model_path"])
    except Exception as e:
        logging.warning(f"Buy model or scaler missing for {config['symbol']}: {e}")
        print(f"Buy model or scaler missing for {config['symbol']}: {e}")
    try:
        sellscaler = load(config["sell_scaler_path"])
        sellmodel = load(config["sell_model_path"])
    except Exception as e:
        logging.warning(f"Sell model or scaler missing for {config['symbol']}: {e}")
        print(f"Sell model or scaler missing for {config['symbol']}: {e}")
    
    with open(config["features_path"], 'r') as f:
        feature_names = json.load(f)
    buy_features = feature_names[:-1]
    sell_features = feature_names[:-1]

    last_candle = fetch_latest_candle(config["symbol"], config["timeframe"])
    if last_candle is None:
        logging.error(f"Failed to fetch initial candle data for {config['symbol']}.")
        print(f"Failed to fetch initial candle data for {config['symbol']}.")
        return

    try:
        while True:
            current_candle = fetch_latest_candle(config["symbol"], config["timeframe"])
            if current_candle and current_candle['open'] != last_candle['open']:
                if buyscaler is not None and buymodel is not None:
                    predict_and_trade(
                        buyscaler, buymodel, config["symbol"], config["volume"], config["timeframe"],
                        config["buy_risk_reward_ratio"], config["mean_candle_size"], utc_from, utc_to, buy_features, "Buy",
                        threshold=config.get("buy_threshold", None)
                    )
                if sellscaler is not None and sellmodel is not None:
                    predict_and_trade(
                        sellscaler, sellmodel, config["symbol"], config["volume"], config["timeframe"],
                        config["sell_risk_reward_ratio"], config["mean_candle_size"], utc_from, utc_to, sell_features, "Sell",
                        threshold=config.get("sell_threshold", None)
                    )
                last_candle = current_candle
            time.sleep(60)
    except KeyboardInterrupt:
        logging.info(f"Script terminated by user for {config['symbol']}.")
        print(f"Script terminated by user for {config['symbol']}.")
    finally:
        mt5.shutdown()
        logging.info(f"MetaTrader 5 connection closed for {config['symbol']}.")
        print(f"MetaTrader 5 connection closed for {config['symbol']}.")

def main():
    config = {'login': 51708234, 'password': '4bM&wuVJcBTnjV', 'server': 'ICMarketsEU-Demo'}
    init_mt5_connection(config['login'], config['password'], config['server'])
    
    usdcad_config = {
        "symbol": "USDCAD",
        "timeframe": mt5.TIMEFRAME_D1,
        "volume": 0.1,
        "mean_candle_size": 0.088,
        "buy_risk_reward_ratio": "2:3",
        "sell_risk_reward_ratio": "2:3",
        "buy_scaler_path": 'USDCAD_D1_3112_BuyNEW/scaler.joblib',
        "buy_model_path": 'USDCAD_D1_3112_BuyNEW/model.joblib',
        "sell_scaler_path": 'USDCAD_D1_3112_Sell/scaler.joblib',
        "sell_model_path": 'USDCAD_D1_3112_Sell/model.joblib',
        "features_path": 'USDCAD_D1_3112_Sell/feature_names.json'
    }

    utc_from = datetime(2023, 5, 1, tzinfo=pytz.utc)
    utc_to = datetime.now(pytz.utc)

    process_pair(usdcad_config, utc_from, utc_to)

if __name__ == "__main__":
    main()


c:\ProgramData\anaconda3\envs\forex_env\Lib\site-packages\dask\dataframe\_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


Connected to MetaTrader 5
